<a href="https://colab.research.google.com/github/Sidhtang/30-day-ml-challenge-/blob/main/day_14text_generation_using_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#autoregressive or causal language models, GPT-2 is pretrained to estimate
#the probability Pᅵᅴ of a sequence of tokens ᅵ = y1
#, y2
#, ...yt
# occurring in the text,
#given some initial prompt or context sequence ᅴ = x1
, x2
, ...xk
#. Since it is impractical
#to acquire enough training data to estimate Pᅵᅴ directly, it is common to use the
#chain rule of probability to factorize it as a product of conditional probabilities:


'...xk'

In [3]:
#we append it to the prompt and then use
#the new input sequence to generate another token. We do this until we have reached a
#special end-of-sequence token or a predefined maximum length.
#ince the output sequence is conditioned on the choice of input
#prompt, this type of text generation is often called conditional text
#generation.r we’ll explore a few
#of these approximations and gradually build up toward smarter and more complex
#algorithms that can be used to generate high-quality texts.

In [4]:
# @title  Greedy Search Decoding
#The simplest decoding method to get discrete tokens from a model’s continuous out‐
#put is to greedily select the token with the highest probability at each timestep:
#Transformers are the” as the input
#prompt and run the decoding for eight timesteps. At each timestep, we pick out the
#####ernatives:


SyntaxError: invalid character '”' (U+201D) (<ipython-input-4-20c3e79eda4c>, line 4)

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
import pandas as pd
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5
with torch.no_grad():
 for _ in range(n_steps):
  # Indent the following lines to include them in the for loop
  iteration = dict()
  iteration["Input"] = tokenizer.decode(input_ids[0])
  output = model(input_ids=input_ids)
  # Select logits of the first batch and the last token and apply softmax
  next_token_logits = output.logits[0, -1, :]
  next_token_probs = torch.softmax(next_token_logits, dim=-1)
  sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
  # Store tokens with highest probabilities
  for choice_idx in range(choices_per_step):
   token_id = sorted_ids[choice_idx]
   token_prob = next_token_probs[token_id].cpu().numpy()
   token_choice = (
    f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
   )
   iteration[f"Choice {choice_idx+1}"] = token_choice
  # Append predicted next token to input
  input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
  iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.29%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.27%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in ...,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


In [ ]:
# other way is by using geearate function
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

In [5]:
#reproducing the unicron story with openAI

In [9]:
max_length = 128
input_txt = """
In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains. Even more surprising to the
researchers was the fact that the unicorns spoke perfect English.
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains. Even more surprising to the
researchers was the fact that the unicorns spoke perfect English.
The researchers, from the University of California, Santa Barbara, were
surprised to find the unicorns, which are known as the "Ursa Major" in Latin,
were living in the remote valley, which is located in the Andes Mountains,
about 1,000 miles from the nearest city.
"We were surprised to find the unicorns


In [ ]:
# @title Beam Search Decoding
# the con of greedy search decoding is that it tendss to produce repetitive
#Instead of decoding the token with the highest probability at each step, beam search
#keeps track of the top-b most probable next tokens, where b is referred to as the num‐
#er of beams or partial hypotheses process is repeated until we reach the maximum length.

In [ ]:
#Let’s calculate and compare the log probabilities of the texts generated by greedy and
#beam search to see if beam search can improve the overall probability.
import torch.nn.functional as F
def log_probs_from_logits(logits, labels):
 logp = F.log_softmax(logits, dim=-1)
 logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
 return logp_label
 # now for the sequence part cwe just need to sum the proabilities
def sequence(model,labels,input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(output.logits[:,:-1,:],labels[:,1:])
    seq_log_probs = torch.sum(log_probs[:,input_len:])
  return seq_log_probs.cpu().numpy()


In [ ]:
#calculating the greedy decoder probability
logp = sequence(model,output_greedy,input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

In [ ]:
# @title beam probability
# @title beam probability
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
 do_sample=False)
# Use 'sequence' instead of 'sequence_logprob'
logp = sequence(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

In [ ]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
 do_sample=False, no_repeat_ngram_size=2)
logp = sequence(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")
# to avoid repetive search in beam algorithm we  use n gram penalty

In [ ]:
# @title sampling methods
#simplest sampling method is to  randomly sample form the probaility disribuiton of the model
# effect of temperature  on the token probabilities



In [ ]:
#To see how we can use temperature to influence the generated text, let’s sample with
#3T= 2 by setting the temperature parameter in the generate() function (we’ll
#explain the meaning of the top_k parameter in the next section)
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
 temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))


In [ ]:
#We can clearly see that a high temperature has produced mostly gibberish; by accen‐
#tuating the rare tokens, we’ve caused the model to create strange grammar and quite a
#few made-up words
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
 temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
#main lesson is that temperature   produces a lot of variety in text genearation
#Another way to adjust the trade-off between coherence and diversity is to truncate
#the distribution of the vocabulary


In [ ]:
# @title Top-k and Nucleus Sampling
#Top-k and nucleus (top-p) sampling are two popular alternatives or extensions to
#using temperature. In both cases, the basic idea is to restrict the number of possible
#tokens we can sample from at each timestep
#The idea behind top-k sampling is to avoid the low-probability choices by only sam‐
#pling from the k tokens with the highest probability.
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,
 top_k=50)
print(tokenizer.decode(output_topk[0]))


In [ ]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
 top_p=0.90)
print(tokenizer.decode(output_topp[0]))

In [ ]:
# If you want
#your model to perform a precise task like arithmetic or providing an answer to a spe‐
#cific question, then you should lower the temperature or use deterministic methods
#like greedy search in combination with beam search to guarantee getting the most
#likely answer. If you want the model to generate longer texts and even be a bit crea‐
#tive, then you should switch to sampling methods and increase the temperature or
#use a mix of top-k and nucleus sampling.

In [15]:
# @title Chapteer 6 summarisation
#we will be looking at a cnn datset for daily mail

#to build a summariser model
!pip install datasets
from datasets import load_dataset # Corrected import statement

dataset = load_dataset("cnn_dailymail", version ='3.0.0') # Corrected variable name to avoid overwriting the 'datasets' module
print(f"Features: {dataset['train'].column_names}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [16]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


NameError: name 'dataset' is not defined

In [ ]:
#t. The standard, yet crude way to deal with this
#for summarization is to simply truncate the texts beyond the model’s context size.
#Obviously there could be important information for the summary toward the end of
#the text

In [11]:
#text summerisation pipelines
#we will look
sample_text = dataset["train"][1]["article"][:2000]
# We'll collect the generated summaries of each model in a dictionary
summaries = {}
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)











NameError: name 'dataset' is not defined